# Speed Restrictions by Day Data Processing

### Columns to drop:
1. Location_Description
2. Direction_Sort
3. Restriction_Distance_Miles
4. Line_Total_Track_Miles
5. Systemwide_Total_Track_Miles

In [ ]:
dataframes = []
for i in range(1,13):
    speed_restrictions_filename_template:str = f'data/2023-{i:0>2}_Speed_Restrictions_By_Day.csv'
    month_df = pd.read_csv(speed_restrictions_filename_template)

    # Drop specified columns
    month_df.drop(labels=['Location_Description', 'Direction_Sort', 'Restriction_Distance_Miles', 'Line_Total_Track_Miles', 'Systemwide_Total_Track_Miles'], axis=1, inplace=True)

    # add dataframe to list
    dataframes.append(month_df)

speed_restrictions_2023_df:pd.DataFrame = pd.concat(dataframes)
speed_restrictions_2023_df.drop('SRV_MAIN_UNIQUE_ID', axis=1, inplace=True)
speed_restrictions_2023_df
    

,Calendar_Date,ID,Track_Direction,Line,Branch,Track_Name,Loc_GTFS_Stop_ID,Location_Type,Restriction_Status,Date_Restriction_Reported,...,SR_Restriction_Distance_Span,Restriction_Path,Restriction_Days_Active_On_Calendar_Day,Restriction_Days_to_Clear,Daily_Restriction_Count_Start,Month_Restriction_Count_Start,Restriction_Count_New,Restriction_Count_Cleared,Month_Restriction_Count_End,Daily_Restriction_Count_End
0,2023-01-01,329396,EB,Green Line,Green Line Trunk,GL Kenmore-College Ave EB,place-armnl | place-boyls,Between Stations,Active Restriction,2021-09-22,...,Single Segment,Start|End,466,NaN,1,1,0,0,0,1
1,2023-01-01,334870,EB,Green Line,Green Line Trunk,GL Kenmore-College Ave EB,place-north | place-spmnl,Between Stations,Active Restriction,2021-10-05,...,Single Segment,Start|End,453,NaN,1,1,0,0,0,1
2,2023-01-01,443512,EB,Green Line,Green Line Trunk,GL Kenmore-College Ave EB,place-north | place-spmnl,Between Stations,Active Restriction,2022-07-12,...,Single Segment,Start|End,173,NaN,1,1,0,0,0,1
3,2023-01-01,358277,EB,Green Line,Green Line Trunk,GL Kenmore-College Ave EB,place-spmnl | place-lech,Between Stations,Active Restriction,2021-12-17,...,Single Segment,Start|End,380,NaN,1,1,0,0,0,1
4,2023-01-01,358285,WB,Green Line,Green Line Trunk,GL Kenmore-College Ave WB,place-lech | place-spmnl,Between Stations,Active Restriction,2021-12-17,...,Single Segment,Start|End,380,NaN,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6739,2023-12-05,570802,NB,Orange Line,Orange Line,OL NB,place-forhl | place-grnst,Between Stations,Active Restriction,2023-08-22,...,Single Segment,Start|End,105,NaN,1,0,0,0,0,1
6740,2023-12-04,570802,NB,Orange Line,Orange Line,OL NB,place-forhl | place-grnst,Between Stations,Active Restriction,2023-08-22,...,Single Segment,Start|End,104,NaN,1,0,0,0,0,1
6741,2023-12-03,570802,NB,Orange Line,Orange Line,OL NB,place-forhl | place-grnst,Between Stations,Active Restriction,2023-08-22,...,Single Segment,Start|End,103,NaN,1,0,0,0,0,1
6742,2023-12-02,570802,NB,Orange Line,Orange Line,OL NB,place-forhl | place-grnst,Between Stations,Active Restriction,2023-08-22,...,Single Segment,Start|End,102,NaN,1,0,0,0,0,1


### Compressing Speed Restrictions by Restriction ID

In [ ]:
sr_2023_compressed_start_date = speed_restrictions_2023_df.groupby(['ID', 'Loc_GTFS_Stop_ID'])['Calendar_Date'].min().reset_index()
sr_2023_compressed_start_date['start_date'] = sr_2023_compressed_start_date['Calendar_Date']

sr_2023_compressed_end_date = speed_restrictions_2023_df.groupby(['ID', 'Loc_GTFS_Stop_ID'])['Calendar_Date'].max().reset_index()
sr_2023_compressed_end_date['end_date'] = sr_2023_compressed_end_date['Calendar_Date']

# sr_2023_compressed_total_days_active = speed_restrictions_2023_df.groupby(['ID', 'Loc_GTFS_Stop_ID'])['Restriction_Days_Active_On_Calendar_Day'].max()
# sr_2023_compressed_total_days_cleared =  

sr_2023_compressed = speed_restrictions_2023_df.groupby(['ID', 'Loc_GTFS_Stop_ID'])[['Restriction_Status', 'Restriction_Reason', 'Track_Direction', 'Line', 'Branch', 'Track_Name', 'Location_Type', 'Restriction_Speed_MPH', 'Restriction_Distance_Feet', 'Line_Restricted_Track_Pct', 'Systemwide_Restricted_Track_Pct', 'SR_Restriction_Distance_Span', 'Restriction_Path']]
sr_2023_compressed = sr_2023_compressed.apply(pd.DataFrame)

sr_2023_compressed_start_end = sr_2023_compressed_start_date.merge(sr_2023_compressed_end_date, left_on=['ID', 'Loc_GTFS_Stop_ID'], right_on=['ID', 'Loc_GTFS_Stop_ID'])
sr_2023_compressed_start_end.drop(['Calendar_Date_x', 'Calendar_Date_y'], axis=1, inplace=True)
sr_2023_compressed = sr_2023_compressed.merge(sr_2023_compressed_start_end, left_on=['ID', 'Loc_GTFS_Stop_ID'], right_on=['ID', 'Loc_GTFS_Stop_ID'])
sr_2023_compressed

,ID,Loc_GTFS_Stop_ID,Restriction_Status,Restriction_Reason,Track_Direction,Line,Branch,Track_Name,Location_Type,Restriction_Speed_MPH,Restriction_Distance_Feet,Line_Restricted_Track_Pct,Systemwide_Restricted_Track_Pct,SR_Restriction_Distance_Span,Restriction_Path,start_date,end_date
0,20,place-aport,Active Restriction,NaN,WB,Blue Line,Blue Line,BL WB,Station,10 mph,101.0,0.001533,0.000140,Multi-Segment,End,2023-10-17,2023-10-19
1,20,place-aport,Restriction Cleared,NaN,WB,Blue Line,Blue Line,BL WB,Station,10 mph,101.0,0.001533,0.000140,Multi-Segment,End,2023-10-17,2023-10-19
2,20,place-aport,Restriction Opened,NaN,WB,Blue Line,Blue Line,BL WB,Station,10 mph,101.0,0.001533,0.000140,Multi-Segment,End,2023-10-17,2023-10-19
3,20,place-wimnl | place-aport,Active Restriction,NaN,WB,Blue Line,Blue Line,BL WB,Between Stations,10 mph,699.0,0.010608,0.000970,Multi-Segment,Start,2023-10-17,2023-10-19
4,20,place-wimnl | place-aport,Restriction Cleared,NaN,WB,Blue Line,Blue Line,BL WB,Between Stations,10 mph,699.0,0.010608,0.000970,Multi-Segment,Start,2023-10-17,2023-10-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91533,578017,place-nqncy | place-jfk,Active Restriction,Track,NB,Red Line,Red Line - Braintree,RL Alewife-Braintree NB,Between Stations,25 mph,300.0,0.001198,0.000416,Single Segment,Start|End,2023-09-22,2023-12-31
91534,578017,place-nqncy | place-jfk,Active Restriction,Track,NB,Red Line,Red Line - Braintree,RL Alewife-Braintree NB,Between Stations,25 mph,300.0,0.001198,0.000416,Single Segment,Start|End,2023-09-22,2023-12-31
91535,578017,place-nqncy | place-jfk,Active Restriction,Track,NB,Red Line,Red Line - Braintree,RL Alewife-Braintree NB,Between Stations,25 mph,300.0,0.001198,0.000416,Single Segment,Start|End,2023-09-22,2023-12-31
91536,578017,place-nqncy | place-jfk,Active Restriction,Track,NB,Red Line,Red Line - Braintree,RL Alewife-Braintree NB,Between Stations,25 mph,300.0,0.001198,0.000416,Single Segment,Start|End,2023-09-22,2023-12-31


In [ ]:
# speed_restrictions_2023_df.groupby(['ID', 'Loc_GTFS_Stop_ID'])['Calendar_Date'].max()
sr_2023_compressed_start_end

,ID,Loc_GTFS_Stop_ID,start_date,end_date
0,20,place-aport,2023-10-17,2023-10-19
1,20,place-wimnl | place-aport,2023-10-17,2023-10-19
2,21,place-mvbcl | place-aport,2023-12-14,2023-12-31
3,43,place-astao | place-welln,2023-10-03,2023-10-12
4,44,place-rugg,2023-10-17,2023-12-21
...,...,...,...,...
724,577753,place-mdftf | place-balsq,2023-09-20,2023-10-10
725,577754,place-balsq | place-mgngl,2023-09-20,2023-10-10
726,577755,place-gilmn | place-esomr,2023-09-20,2023-10-10
727,577756,place-gilmn | place-esomr,2023-09-20,2023-10-10
